# Evals

An Eval is a way to define a test scenario of one or more prompts. (Multiple (sequential) prompts can be used to evaluate conversations where the LLM (i.e. the underlying client) maintains a conversational history.) The response of the LLM is evaluated based on the "checks" that are defined for that Eval. Evals can be defined with a dictionary/yaml, or with llm_eval python classes.

Here's a simple example where we want to test the LLM's ability to create a python function that we've described. (A more detailed version of this Eval can be found in `./evals/mask_emails.yaml`)

## Defining a single Eval and single prompt.

In [1]:
from llm_eval.eval import Eval, PromptTest
from llm_eval.checks import RegexCheck, PythonCodeBlockTests

eval_obj = Eval(
    metadata={
        # metadata is a dictionary that can contain any key-value string pairs
        'name': 'Mask Emails',
        'description': 'Evaluates the ability of a model to mask emails in text',
    },
    prompt_sequence=[
        PromptTest(
            prompt="""
            Create a python function called `mask_emails` that takes a single string and masks all
            emails in that string.

            For each email in the format of `x@y.z`, the local part (`x`) should be masked with
            [MASKED].
            """,
            checks=[
                RegexCheck(pattern=r'def mask_emails\([a-zA-Z_]+(\: str)?\)( -> str)?\:'),
                PythonCodeBlockTests(
                    code_tests=[
                        'assert mask_emails("no email") == "no email"',
                        'assert mask_emails("my email is a@b.c") == "my email is [MASKED]@b.c"',
                    ],
                ),
            ],
        ),
    ],
)


/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The `PythonCodeBlockTests` check object extracts all of the code blocks that were generated by the LLM and runs the code in the background in an isolated environment. It tracks the number of code blocks that were generated and the number of code blocks that successfully executed. The object also has a set of `code_test` where the user can write python code (either single statements (assertion or statements that resolve to booleans) or functions (which return boolean values)). The code that is defined in these `code_tests` is run in the same isolated enviornment and can test any function, variable, or class that is created from code generated by the LLM.

See `./evals/mask_emails.yaml` for additional examples, and the documentation for `PythonCodeBlockTests` in `llm_eval/checks.py`.

The eval can also be defined as a dictionary (or, for example, loaded from a yaml file into a dictionary) as shown below.

In [2]:
eval_dict = {
    'metadata': {
        'name': 'Mask Emails',
        'description': 'Evaluates the ability of a model to mask emails in text',
    },
    'prompt_sequence': [
        {
            'prompt':
                """
                Create a python function called `mask_emails` that takes a single string and masks all
                emails in that string.

                For each email in the format of `x@y.z`, the local part (`x`) should be masked with
                [MASKED].
                """,
            'checks': [
                {
                    'check_type': 'REGEX',
                    'pattern': 'def mask_emails\\([a-zA-Z_]+(\\: str)?\\)( -> str)?\\:',
                },
                {
                    'check_type': 'PYTHON_CODE_BLOCK_TESTS',
                    'code_tests': [
                        'assert mask_emails("no email") == "no email"',
                        'assert mask_emails("my email is a@b.c") == "my email is [MASKED]@b.c"',
                    ],
                },
            ],
        },
    ],
}

In the Eval above, we can define checks with dictionaries using the `check_type` key and values like `REGEX` and `PYTHON_CODE_BLOCK_TESTS`, which are "registered" so the classes can be instantiated in real time. Built-in registration types can be found in the `CheckType` enum.

For example, here is the class definition for `RegexCheck`

```
@Check.register(CheckType.REGEX)
class RegexCheck(Check):
    ...
```

Users can create their own custom checks and use the same registration system to register their checks, for example:

```
@Check.register('my-custom-check')
class CustomXYZCheck(Check):
    ...
```


Eval objects can be created directly from dictionaries, as shown below. However, the most common pattern is to define Evals as yaml files, and load many evals and run them with the `EvalHarness`, which will be described in another notebook.

In [3]:
Eval(**eval_dict).to_dict()

{'metadata': {'name': 'Mask Emails',
  'description': 'Evaluates the ability of a model to mask emails in text'},
 'prompt_sequence': [{'prompt': 'Create a python function called `mask_emails` that takes a single string and masks all\nemails in that string.\n\nFor each email in the format of `x@y.z`, the local part (`x`) should be masked with\n[MASKED].\n',
   'checks': [{'pattern': 'def mask_emails\\([a-zA-Z_]+(\\: str)?\\)( -> str)?\\:',
     'check_type': 'REGEX'},
    {'code_tests': ['assert mask_emails("no email") == "no email"',
      'assert mask_emails("my email is a@b.c") == "my email is [MASKED]@b.c"'],
     'check_type': 'PYTHON_CODE_BLOCK_TESTS'}]}]}

## Using multiple (sequential prompts)

Multiple (sequential) prompts can be used to evaluate conversations where the LLM (i.e. the underlying client) maintains a conversational history.

For example, if we're testing the ability of an LLM to generate a function that masks emails, a logical next step (that someone using the LLM for a similar use case) is to create unit tests or assertion statements that test the accuracy of the function.

Let's modify our original Eval to define another prompt and set of checks that includes this followup request.

Here is the modification to the Eval defined with classes:

In [4]:
from llm_eval.eval import Eval, PromptTest
from llm_eval.checks import MatchCheck, RegexCheck, PythonCodeBlocksPresent, PythonCodeBlockTests

eval_obj = Eval(
    metadata={
        # metadata is a dictionary that can contain any key-value string pairs
        'name': 'Mask Emails',
        'description': 'Evaluates the ability of a model to mask emails in text',
    },
    prompt_sequence=[
        PromptTest(
            prompt="""
            Create a python function called `mask_emails` that takes a single string and masks all
            emails in that string.

            For each email in the format of `x@y.z`, the local part (`x`) should be masked with
            [MASKED].
            """,
            checks=[
                RegexCheck(pattern=r'def mask_emails\([a-zA-Z_]+(\: str)?\)( -> str)?\:'),
            ],
        ),
        PromptTest(
            prompt="Create a set of assertion statements that test the function.",
            checks=[
                MatchCheck(value="assert "),
                PythonCodeBlockTests(
                    code_tests=[
                        'assert mask_emails("no email") == "no email"',
                        'assert mask_emails("my email is a@b.c") == "my email is [MASKED]@b.c"',
                    ],
                ),
            ],
        ),
    ],
)


We've included a second `PromptTest` object. It's now a more clear that the PromptTest class is a way to define a single prompt and list of checks to test that specific prompt. We've also added a `MatchCheck` which ensures the response contains at least one assertion statement.

**NOTE**: One important caveat is that we've moved `PythonCodeBlockTests` to the 2nd/last PromptTest. The reason for this (also explained in the class documentation) is that this check runs all of the code blocks generated across all responses. Therefore, this check should only be included once, on the last PromptTest object.

Here is the equivalent change in our the dictionary version of our eval.

In [5]:
eval_dict = {
    'metadata': {
        'name': 'Mask Emails',
        'description': 'Evaluates the ability of a model to mask emails in text',
    },
    'prompt_sequence': [
        {
            'prompt':
                """
                Create a python function called `mask_emails` that takes a single string and masks all
                emails in that string.

                For each email in the format of `x@y.z`, the local part (`x`) should be masked with
                [MASKED].
                """,
            'checks': [
                {
                    'check_type': 'REGEX',
                    'pattern': 'def mask_emails\\([a-zA-Z_]+(\\: str)?\\)( -> str)?\\:',
                },
            ],
        },
        {
            'prompt':" Create a set of assertion statements that test the function.",
            'checks': [
                {
                    'check_type': 'MATCH',
                    'value': 'assert ',
                },
                {
                    'check_type': 'PYTHON_CODE_BLOCK_TESTS',
                    'code_tests': [
                        'assert mask_emails("no email") == "no email"',
                        'assert mask_emails("my email is a@b.c") == "my email is [MASKED]@b.c"',
                    ],
                },
            ],
        },
    ],
}

---

## Creating Candidates

A candidate is just a wrapper around an LLM/service that has a similar registration system (as Checks) that allows Candidates to be instantiated dynamically from dictionaries. 

In [6]:
from llm_eval.candidates import OpenAICandidate

candidate = OpenAICandidate(
    metadata={'name': 'OpenAI GPT-3.5-Turbo (0125)'},
    parameters={
        'model_name': 'gpt-3.5-turbo-0125',
        'system_message': "You are a helpful AI assistant.",
        'temperature': 0.1,
    },
)

The equivalent dictionary representation is:

In [7]:
candidate_dict = {
    'metadata': {'name': 'OpenAI GPT-3.5-Turbo (0125)'},
    'candidate_type': 'OPENAI',
    'parameters': {
        'model_name': 'gpt-3.5-turbo-0125',
        'system_message': "You are a helpful AI assistant.",
        'temperature': 0.1,
    },
}

In [8]:
assert OpenAICandidate.from_dict(candidate_dict).to_dict() == candidate_dict

---

# Running a single Eval against a single Candidate

This example shows running a single Eval against a Eingle candidate. However, as mentioned above, the most common pattern is to define Evals as yaml files, and load many evals and run them with the `EvalHarness`, which will be described in another notebook.

The Eval object is callable, and takes a single candidate.

In [9]:
result = eval_obj(candidate)
print(result)

EvalResult:
    Candidate:                  OpenAI GPT-3.5-Turbo (0125)
    Eval:                        Mask Emails
    # of Prompts Tested:         2
    Cost:                       $0.0007
    Total Response Time:         9.5 seconds
    # of Response Characters:    1,510
    Characters per Second:       159.3
    # of Checks:                 3
    # of Successful Checks:      1
    % of Successful Checks:      33.3%
    # of Code Blocks Generated:  2
    # of Successful Code Blocks: 2
    # of Code Tests Defined:     2
    # of Successful Code Tests:  1


In [10]:
type(result)

llm_eval.eval.EvalResult

The results of the Checks are stored in a `results` property of the `EvalResult`. The `results` property is a list of lists. Each outer lists corresponds to each (sequential) prompt we tested. Our Eval object had two prompts (PromptTests). The first was a request to generate the `mask_emails` function. The second was a request to create a set of assertion statements to test the function. 

Therefore, `result.results` will be a list of length `2`.

Each item in the list is a list of CheckResult objects corresponding to the Checks in the PromptTest. The first test had one check (RegexCheck), the second had two checks (MatchCheck, and PythonCodeBlockTests).

Therefore, `result.results[0]` will be a list of `1` CheckResult objects, and `result.results[1]` will be a list of `2` `CheckResult` objects.

In [11]:
print(f"Number of lists (which correspond to the number of prompts testsed): {len(result.results)}")
print(f"Number of CheckResults associated with the first prompt: {len(result.results[0])}")
print(f"Number of CheckResults associated with the second prompt: {len(result.results[1])}")

Number of lists (which correspond to the number of prompts testsed): 2
Number of CheckResults associated with the first prompt: 1
Number of CheckResults associated with the second prompt: 2


---

In [12]:
result.results[0][0].to_dict()

{'value': True,
 'success': True,
 'metadata': {'check_type': 'REGEX',
  'check_pattern': 'def mask_emails\\([a-zA-Z_]+(\\: str)?\\)( -> str)?\\:',
  'check_negate': False,
  'check_metadata': {}},
 'result_type': 'PASS_FAIL'}

In [13]:
result.results[0][0].metadata

{'check_type': 'REGEX',
 'check_pattern': 'def mask_emails\\([a-zA-Z_]+(\\: str)?\\)( -> str)?\\:',
 'check_negate': False,
 'check_metadata': {}}

In [14]:
result.results[1][1].metadata

{'check_type': 'PYTHON_CODE_BLOCK_TESTS',
 'num_code_blocks': 2,
 'num_code_blocks_successful': 2,
 'code_blocks': ['import re\n\ndef mask_emails(input_string):\n    email_pattern = r\'\\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\\.[A-Z|a-z]{2,}\\b\'\n    masked_string = re.sub(email_pattern, \'[MASKED]@[MASKED].[MASKED]\', input_string)\n    return masked_string\n\n# Test the function\ninput_string = "Please send the report to john.doe@example.com and jane.smith@example.org"\nmasked_output = mask_emails(input_string)\nprint(masked_output)',
  '# Test cases\nassert mask_emails("Please send the report to john.doe@example.com") == "Please send the report to [MASKED]@[MASKED].[MASKED]"\nassert mask_emails("Contact us at info@example.org for more information") == "Contact us at [MASKED]@[MASKED].[MASKED] for more information"\nassert mask_emails("Emails: alice@example.com, bob@example.net") == "Emails: [MASKED]@[MASKED].[MASKED], [MASKED]@[MASKED].[MASKED]"\nassert mask_emails("No emails in this st

In [15]:
len(result.all_check_results)

3

In [16]:
result.all_check_results[-1].to_dict()

{'value': 0.75,
 'success': False,
 'metadata': {'check_type': 'PYTHON_CODE_BLOCK_TESTS',
  'num_code_blocks': 2,
  'num_code_blocks_successful': 2,
  'code_blocks': ['import re\n\ndef mask_emails(input_string):\n    email_pattern = r\'\\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\\.[A-Z|a-z]{2,}\\b\'\n    masked_string = re.sub(email_pattern, \'[MASKED]@[MASKED].[MASKED]\', input_string)\n    return masked_string\n\n# Test the function\ninput_string = "Please send the report to john.doe@example.com and jane.smith@example.org"\nmasked_output = mask_emails(input_string)\nprint(masked_output)',
   '# Test cases\nassert mask_emails("Please send the report to john.doe@example.com") == "Please send the report to [MASKED]@[MASKED].[MASKED]"\nassert mask_emails("Contact us at info@example.org for more information") == "Contact us at [MASKED]@[MASKED].[MASKED] for more information"\nassert mask_emails("Emails: alice@example.com, bob@example.net") == "Emails: [MASKED]@[MASKED].[MASKED], [MASKED]@[MASKED]